In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/project/extra_flowers.zip'

In [ ]:
pip install ultralytics

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import os
import zipfile
from tqdm import tqdm
import shutil
import yaml
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import torch

In [ ]:
def clean_unzip(zip_path, output_dir):
    """Распаковывает архив, игнорируя технические файлы Mac и лишнюю вложенность"""
    os.makedirs(output_dir, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Фильтрация: только нужные файлы (images/, labels/)
        valid_files = [
            f for f in zip_ref.namelist()
            if (f.startswith('extra_flowers/images/')
                or f.startswith('extra_flowers/labels/'))
            and not ('__MACOSX' in f or '._' in f or '.DS_Store' in f)
        ]

        # Распаковка с прогресс-баром
        for file in tqdm(valid_files, desc="Распаковка"):
            try:
                # Удаляем 'extra_flowers/' из пути
                dest_path = file.replace('extra_flowers/', '')
                # Полный путь для сохранения
                full_path = os.path.join(output_dir, dest_path)

                # Создаем папки, если их нет
                os.makedirs(os.path.dirname(full_path), exist_ok=True)

                # Извлекаем только если это файл (не папка)
                if not file.endswith('/'):
                    with zip_ref.open(file) as src, open(full_path, 'wb') as dst:
                        dst.write(src.read())
            except Exception as e:
                print(f"Ошибка с {file[:50]}...: {str(e)[:50]}")

# Запуск
clean_unzip('/content/drive/MyDrive/project/extra_flowers.zip', '/content/extra_flowers')

# Дополнительная очистка
!find clean_data -name '__MACOSX' -exec rm -rf {} + 2>/dev/null
!find clean_data -name '.DS_Store' -delete 2>/dev/null
!find clean_data -name '._*' -delete 2>/dev/null

print("Готово! Структура:")
!tree clean_data -L 2

Распаковка: 100%|██████████| 1473/1473 [00:02<00:00, 652.56it/s] 


Готово! Структура:
/bin/bash: line 1: tree: command not found


In [ ]:
#!rm -r extra_flowers/

In [ ]:
# ======================
# 1. НАСТРОЙКИ ПУТЕЙ
# ======================
DATA_SOURCE = "/content/extra_flowers"
DATA_DEST = "/content/datasets/extra_flowers"
CLASS_NAMES = [
    "Bellflower", "Black-eyed Susan", "Calendula", "California poppy",
    "Carnation", "Daffodil", "Daisy", "Dandelion", "Iris", "Magnolia",
    "Rose", "Sunflower", "Tulip", "Water lily"
]
TRAIN_RATIO = 0.8

# ======================
# 2. СОЗДАЕМ СТРУКТУРУ ПАПОК
# ======================
os.makedirs(f"{DATA_DEST}/extra_flowers/train/images", exist_ok=True)
os.makedirs(f"{DATA_DEST}/extra_flowers/train/labels", exist_ok=True)
os.makedirs(f"{DATA_DEST}/extra_flowers/val/images", exist_ok=True)
os.makedirs(f"{DATA_DEST}/extra_flowers/val/labels", exist_ok=True)

# ======================
# 3. ПРОВЕРКА ИСХОДНЫХ ДАННЫХ
# ======================
def validate_data_structure():
    """Проверяет соответствие изображений и разметок"""
    images = [f for f in os.listdir(f"{DATA_SOURCE}/images")
             if f.lower().endswith(('.jpg', '.png', '.jpeg', '.webp'))]
    missing = []

    for img in images:
        label = os.path.splitext(img)[0] + ".txt"
        if not os.path.exists(f"{DATA_SOURCE}/labels/{label}"):
            missing.append(img)

    if missing:
        print(f"Найдено {len(missing)} изображений без разметки!")
        return [img for img in images if img not in missing]
    return images

valid_images = validate_data_structure()

# ======================
# 4. РАЗДЕЛЕНИЕ НА TRAIN/VAL
# ======================
train_files, val_files = train_test_split(valid_images, train_size=TRAIN_RATIO, random_state=42)

def copy_files(file_list, subset):
    """Копирует файлы с соблюдением структуры YOLO"""
    for img in file_list:
        # Изображения
        shutil.copy(
            f"{DATA_SOURCE}/images/{img}",
            f"{DATA_DEST}/extra_flowers/{subset}/images/{img}"
        )
        # Разметки
        label = os.path.splitext(img)[0] + ".txt"
        if os.path.exists(f"{DATA_SOURCE}/labels/{label}"):
            shutil.copy(
                f"{DATA_SOURCE}/labels/{label}",
                f"{DATA_DEST}/extra_flowers/{subset}/labels/{label}"
            )

copy_files(train_files, "train")
copy_files(val_files, "val")

# ======================
# 5. СОЗДАНИЕ data.yaml
# ======================
data_yaml = {
    "train": f"{DATA_DEST}/extra_flowers/train/images",
    "val": f"{DATA_DEST}/extra_flowers/val/images",
    "nc": len(CLASS_NAMES),
    "names": CLASS_NAMES
}

with open(f"{DATA_DEST}/data.yaml", "w") as f:
    yaml.dump(data_yaml, f)

# ======================
# 6. ПРОВЕРКА РЕЗУЛЬТАТА
# ======================
print(f"\n✅ Готово! Итоговая структура:")
print(f"Общее изображений: {len(valid_images)}")
print(f"Train: {len(train_files)}")
print(f"Val: {len(val_files)}")
print(f"YAML: {DATA_DEST}/data.yaml\n")

# Проверка первых 5 файлов
!ls {DATA_DEST}/extra_flowers/train/images | head -5


✅ Готово! Итоговая структура:
Общее изображений: 735
Train: 588
Val: 147
YAML: /content/datasets/extra_flowers/data.yaml

0010fef3-yz-ylt-dem-magnoliya-g-l.jpg
0138b2a1-40612281740_e65e4e1a65_b.jpg
01d087bf-000036.jpg
027be676-xqU02QOq-bk.jpg
03566105-6478256.jpg


In [ ]:
path = "/content/drive/MyDrive/yolo_results_3/weights/best.pt"
model = YOLO(path).to('cuda')

In [ ]:
train_args = {
    'data': '/content/datasets/extra_flowers/data.yaml',
    'epochs': 20,
    'imgsz': 640,
    'batch': 32,                  # Максимально возможный для вашего GPU
    'device': 'cuda',             # Явное указание GPU
    'lr0': 0.001,                 # Пониженный learning rate для дообучения
    'patience': 15,               # Ранняя остановка при отсутствии улучшений
    'optimizer': 'AdamW',         # Лучше для специфичных классов
    'augment': True,              # Критично для цветов!
    'hsv_h': 0.02,                # Усиленная аугментация оттенков
    'flipud': 0.3,                # Вертикальное отражение
    'name': 'flowers_finetune',
    'exist_ok': True,             # Перезапись предыдущих результатов
    'pretrained': True            # Использовать предобученные веса
}

# Запуск дообучения
results = model.train(**train_args)

Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/extra_flowers/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.3, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/yolo_results_3/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=flowers_finetune, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, p

100%|██████████| 5.35M/5.35M [00:00<00:00, 303MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2487.4±747.2 MB/s, size: 529.8 KB)


train: Scanning /content/datasets/extra_flowers/extra_flowers/train/labels.cache... 588 images, 0 backgrounds, 0 corrupt: 100%|██████████| 588/588 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1058, len(boxes) = 2298. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

      20/20      6.17G      1.146     0.7631      1.194         44        640: 100%|██████████| 19/19 [00:16<00:00,  1.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.48it/s]

                   all        147        483        0.8      0.703      0.777      0.508



20 epochs completed in 0.127 hours.
Optimizer stripped from runs/detect/flowers_finetune/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/flowers_finetune/weights/best.pt, 6.3MB

Validating runs/detect/flowers_finetune/weights/best.pt...
Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.21s/it]


                   all        147        483       0.81      0.697      0.774      0.509
            Bellflower         16         55      0.639      0.345      0.461        0.2
      Black-eyed Susan          4         17      0.873      0.647      0.799      0.375
             Calendula          7         21      0.622      0.476      0.566      0.273
      California poppy          9         35      0.879      0.629      0.801      0.378
             Carnation         14         71      0.771      0.704      0.784      0.413
              Daffodil         12         46      0.836      0.543      0.656      0.291
                 Daisy         18         53      0.793      0.528      0.632      0.316
             Dandelion          6          8      0.887      0.983      0.967      0.788
                  Iris         13         21      0.821       0.81      0.833      0.669
              Magnolia         10         19       0.81      0.899      0.933      0.755
                  Ros

In [ ]:
# Сохранение модели
torch.save(model.state_dict(), 'fine_tuned_flowers.pt')

In [ ]:
cp -r runs/detect/flowers_finetune /content/drive/MyDrive/yolo_results_3

In [ ]:
# Путь к архиву
zip_path = '/content/drive/MyDrive/dataset_flowers/flowers_120.v1i.yolov8.zip'
extract_to = '/content/flowers'

# Распаковка
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Архив распакован в:", extract_to)

Архив распакован в: /content/flowers


In [ ]:
!yolo val model=/content/drive/MyDrive/yolo_results_3/weights/best.pt data=/content/flowers/data.yaml split=test name=test_results save_json=True save_hybrid=True


WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.
Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 981.4±279.5 MB/s, size: 36.4 KB)
val: Scanning /content/flowers/test/labels.cache... 168 images, 1 backgrounds, 0 corrupt: 100% 168/168 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 11/11 [00:06<00:00,  1.60it/s]
                   all        168        196      0.281      0.263      0.201      0.148
            Bellflower         14         18       0.17      0.111     0.0798     0.0593
      Black-eyed Susan         14         19      0.148     0.0234     0.0764     0.0438
             Calendula         15         15          0          0     0.0785     0.0618
      California poppy         13         17      0.317      0.588      0.3

тут проверка на новых картинках

In [ ]:
# Пути
model_path = "/content/drive/MyDrive/yolo_results_3/weights/best.pt"
source_dir = "/content/drive/MyDrive/detect/"
output_dir = "/content/results"  # Новая папка для всех результатов

# Создаем папку для результатов, если её нет
os.makedirs(output_dir, exist_ok=True)

# Список файлов
file_names = [
    "waterlily.jpg", "tulip3.jpg", "sunflower.jpg", "rose.jpg", "magnolia.jpg",
    "iris2.jpg", "dandelion.jpg", "daisy2.jpg", "daffodil.jpg", "carnation.jpg",
    "californiapoppy2.jpg", "calendula.jpg", "blackeye.jpg", "bellflower.jpg"
]

for file_name in file_names:
    source_path = os.path.join(source_dir, file_name)

    if not os.path.exists(source_path):
        print(f"Файл {file_name} не найден, пропускаем...")
        continue

    print(f"\nОбработка {file_name}...")

    # Запуск предсказания с явным указанием папки для сохранения
    !yolo predict model={model_path} source={source_path} save=True project={output_dir} name="predict" exist_ok=True

    # Путь к результату
    output_path = os.path.join(output_dir, "predict", file_name)

    if os.path.exists(output_path):
        img = cv2.imread(output_path)
        cv2_imshow(img)
        print(f"Результат для {file_name} успешно загружен.")
    else:
        print(f"Ошибка: результат для {file_name} не найден.")
        !ls {os.path.join(output_dir, "predict")}  # Проверим содержимое папки

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 1. Подключение Google Диска
drive.mount('/content/drive')

# 2. Пути
target_dir = "/content/drive/MyDrive/project"
os.makedirs(f"{target_dir}/predictions", exist_ok=True)
os.makedirs(f"{target_dir}/metrics", exist_ok=True)

# 3. Копирование предсказаний
if os.path.exists("/content/results/predict"):
    !cp -r /content/results/predict/* "{target_dir}/predictions/"
    print("Предсказания скопированы!")
else:
    print("Папка /content/results/predict не найдена.")

# 4. Копирование метрик
if os.path.exists("/content/runs/detect/flowers_finetune"):
    !cp -r /content/runs/detect/flowers_finetune/* "{target_dir}/metrics/"
    print("Метрики скопированы!")
else:
    print("Папка /content/runs/detect/flowers_finetune не найдена.")

# 5. Итоговая проверка
print("\nИтоговое содержимое папки project:")
!ls -la "{target_dir}"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Предсказания скопированы!
Метрики скопированы!

Итоговое содержимое папки project:
total 291847
-rw------- 1 root root 298791196 May 13 11:56  extra_flowers.zip
drwx------ 3 root root      4096 May 13 14:23  metrics
drwx------ 2 root root      4096 May 13 14:23  predictions
-rw------- 1 root root     51635 May 13 14:22 'Untitled11 (1).ipynb'
